In [11]:
print('================= Welcome Message =================')
print('The program would enable you to import required data from FTP directory into MySQL database.')
print()
print()
print('Program begins.')
print('You would have to enter required date(YYYY/MM/dd) within the following prompt to continue the process.')
print('***Reminder: YYYY contains 4 numbers, while MM and dd contains 2.***')


year = str(input("Please enter YYYY: "))
month = str(input("Please enter mm: "))
day = str(input("Please enter dd: "))

================= Welcome Message =================
The program would enable you to import required data from FTP directory into MySQL database.


Program begins.
You would have to enter required date(YYYY/MM/dd) within the following prompt to continue the process.
***Reminder: YYYY contains 4 numbers, while MM and dd contains 2.***
Please enter YYYY: 2019
Please enter mm: 07
Please enter dd: 05


In [12]:
print()
print()

print('Importing packages needed...')
import os
import ftplib
from ftplib import FTP
import re
import pandas as pd
import sys
from os import listdir
from os.path import isfile, join
import numpy as np

while True:
    try:
        import pymysql
    except BaseException:
        print("Have you installed PyMySQL Package within your local system? If not, install it.")
        print("================= The program has been terminated =================.")
        print("Reminder: Try again after fulfilling the requirements.")
        break
    else:
        break


print('Packages imported successfully.')



Importing packages needed...
Packages imported successfully.


In [13]:
# Change working directory tempData directory
print()
print()

while True:
    try:
        os.chdir('./TempData')
    except BaseException:
        pathnow = os.getcwd()
        print('Working directory now:', pathnow)
        print("Have you created 'TempData' directory within python working directory? If not, create it.")
        print("================= The program has been terminated =================.")
        print("Reminder: Try again after fulfilling the requirements.")
        break
    else:
        pathnow = os.getcwd()
        print("Python working directory has now been changed to", pathnow)
        print('Program would then continue.')
        break



Python working directory has now changed to C:\Users\1907075\PythonNotebook\TempData
Program would then continue.


In [16]:
# Connect to FTP and print out the directory
print()
print()

while True:
    try:
        print('Connecting to ftp server....')
        ftp = ftplib.FTP('XXX', 'XX', 'XX')
    except BaseException:
        print("Unable to connect to FTP site.")
        print("================= The program has been terminated =================.")
        print("Reminder: Try again after fulfilling the requirements.")
        break
    else:
        pathnow = os.getcwd()
        print("Connected successfully")
        print('Program would then continue.')
        break
        
print()
print()
        
while True:
    try:
        ftp.cwd(year)
        ftp.cwd(month)
        ftp.cwd(day)
        print('Connecting to directory', year,'/', month, '/', day)
        print('Printing out file list within directory...')
        print("File List: ")
        files = ftp.dir()
        print(files)
    except BaseException:
        print('Connecting to directory', year,'/', month, '/', day)
        print("Unable find required directory, check if the values are given correctly.")
        print("================= The program has been terminated =================")
        print("Reminder: Try again after fulfilling the requirements.")
        break
    else:
        pathnow = os.getcwd()
        print("Sucessfully connected")
        print('Program would then continue.')
        break



Connecting to ftp://10.88.26.112/....
Connected successfully
Program would then continue.


Connecting to directory 2019 / 07 / 05
Unable find required directory, check if the values are given correctly.
================= The program has been terminated =================
Reminder: Try again after fulfilling the requirements.


In [17]:
# Append file list within python list
dirlist = []
ftp.dir(dirlist.append)

# Create new list includes only filename
newdirlist = []
for i in range(len(dirlist)):
    newdirlist.append(dirlist[i][56:])
print('File List:')
for i,j in enumerate(newdirlist):
    print(newdirlist[i])

File List:
PythonCode
Python_Package
Result-20190406.xlsx
Result-20190408.xlsx
Result-20190409.xlsx
Result-20190410.xlsx
Result-20190411.xlsx
dccel.log
mrtg-2.17.4.tar.gz
my.log
my.log.1


In [ ]:
TFT_ID = []
CF_ID = []
EQP_ID = []

# Look up required document 'XXXXRealTime.csv$', and 
for i in range(len(newdirlist)):
    string = newdirlist[i]
    regex = re.compile('RealTime.csv$')
    match = regex.search(string)
    if str(match) != 'None':
        TFT_ID.append(newdirlist[i][0:13])
        CF_ID.append(newdirlist[i][17:30])
        EQP_ID.append(newdirlist[i][34:38])
        filename = newdirlist[i]

In [ ]:
# Download required data (to python working directory)
def getFile(ftp, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(filename, 'wb').write)
    except:
        print "Error"

for i, j in enumerate(newdirlist):
    getFile(ftp,j)

In [28]:
# Import downloaded data from local filesystem
fileInDirectory = [f for f in listdir('.') if isfile(join('.', f))]

# Connect to MySQl Database
while True:
    try:
        print('Connecting to MySQl Database....')
        conn = pymysql.connect(host='XXX', port=3306, user='XXXX', passwd='XXXX', db='XXXX')
        cur = conn.cursor()
    except BaseException:
        print("Unable to connect to MySQl Database.")
        print("================= The program has been terminated =================.")
        print("Reminder: Try again after fulfilling the requirements.")
        break
    else:
        print("Sucessfully connected")
        print('Program would then continue.')
        print()
        print()
        print()
        break

Connecting to MySQl Database....
Sucessfully connected
Program would then continue.





In [ ]:
# Data import starts, combine the data and export to MySQL Database
encoding = 'utf-16'
for i,j in enumerate(fileInDirectory):
    for a,b in enumerate(TFT_ID):
        IDName = b
        fileName = j
        # Filter out data matches TFT_ID
        regex = re.compile('RealTime.csv$')
        match = regex.search(fileName)
        # If filename match, then....
        if str(match) != 'None':
            # Import data matched
            importedData = pd.read_csv(filename, encoding=encoding)
            # Select the specific value of each imported file
            selected_TFT_ID = TFT_ID[a]
            selected_CF_ID = CF_ID[a]
            selected_EQP_ID = EQP_ID[a]
            # Append new column/data to imported file
            importedData['TFT_ID'] = selected_TFT_ID
            importedData['CF_ID'] = selected_CF_ID
            importedData['EQP_ID'] = selected_EQP_ID
            # Insert combined pandas dataframe into MySQL Database
            importedData.to_sql(con=conn, name='vas_pressure_trans', if_exists='append', flavor='mysql')
            print(filename,'has been inserted into MySQl database.')
            

In [18]:
# Close FTP Connection

ftp.quit()

print('The program has been successfully executed.')

The program has been successfully executed.
